In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1f}'.format)

In [ ]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]
output_dir = output_path / 'RTP/access'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
transit_jobs_access = pd.read_csv(output_path / 'access/transit_jobs_access.csv', 
                                  usecols=['geography', 'value', 'geography_group'])
walk_bike_jobs_access = pd.read_csv(output_path / 'access/walk_bike_jobs_access.csv', 
                                  usecols=['geography_value', 'jobs_1_mile_walk', 'jobs_3_mile_bike', 'geography_group']).\
                                  rename(columns={'geography_value': 'geography'})

parcel_emp = pl.read_csv(output_path / 'landuse/parcels_urbansim.txt', 
                         separator=' ', 
                         columns=['parcelid','emptot_p']).to_pandas()


# jobs access in equity geographies
equity_geogs = summary_config['equity_geogs_comb']

## Jobs Accessible within 45 Minutes of Transit

In [4]:
def process_access_data(jobs_access):
    df_access = jobs_access.copy()
    # rename region
    df_access.loc[jobs_access['geography_group'] == 'region', 'geography'] = 'Region'
    # rename rgc
    df_access.loc[jobs_access['geography_group'] == 'rgc_binary', 'geography'] = ['Not in RGC', 'In RGC']

    df_access_equity_geogs = df_access.loc[
        (df_access['geography_group'].isin(equity_geogs)) & (df_access['geography']=='1')].copy()
    df = df_access.loc[
        (df_access['geography_group'].isin(equity_geogs)) & (df_access['geography']=='0')].copy()

    df_access_equity_geogs['geography'] = df_access_equity_geogs['geography_group']
    df['geography'] = "NOT in " + df['geography_group']

    df_access_equity_geogs = pd.concat([df_access_equity_geogs, df], ignore_index=True)
    df_access_equity_geogs['geography_group'] = 'Equity Geography'

    return df_access, df_access_equity_geogs


df_access_t, df_access_equity_t = process_access_data(transit_jobs_access)
df_access_bp, df_access_equity_bp = process_access_data(walk_bike_jobs_access)
tot_jobs = parcel_emp['emptot_p'].sum()

In [5]:
def job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin([geog, 'region'])].\
        rename(columns={'value': 'Jobs within 45-minute Transit Commute'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs'] = df['Jobs within 45-minute Transit Commute'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')

    return df


In [8]:
df = job_access_geog(df_access_t,'CountyName')

f_name = str(incr)+'_transit_jobs_45min_county_region.csv'
df.to_csv(output_dir / f_name); incr+=1

df

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
df_rgc = job_access_geog(df_access_t,'rgc_binary')
df = job_access_geog(df_access_t,'GrowthCenterName')

df = pd.concat([df_rgc, df.loc[~df.index.isin(['Region','Not in RGC'])]], axis=0)

f_name = str(incr)+'_transit_jobs_45min_rgc.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Region,"136,813.3",6.2%
Not in RGC,"102,529.9",4.7%
In RGC,"459,690.5",21.0%
Auburn,"66,549.1",3.0%
Bellevue,"463,325.5",21.1%
Bothell Canyon Park,"57,691.0",2.6%
Bremerton,"34,260.3",1.6%
Burien,"112,451.1",5.1%
Everett,"80,964.0",3.7%


In [ ]:
df = job_access_geog(df_access_t,'rg_proposed')

f_name = str(incr)+'_transit_jobs_45min_reg_geog.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Region,"136,813.3",6.2%
CitiesTowns,"9,699.0",0.4%
Core,"58,889.3",2.7%
HCT,"46,227.9",2.1%
Metro,"331,867.4",15.1%
UU,"11,570.7",0.5%


In [ ]:
df = job_access_geog(df_access_equity_t,'Equity Geography')

f_name = str(incr)+'_transit_jobs_45min_equity.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
youth_geog_vs_reg_total,"47,806.9",2.2%
elderly_geog_vs_reg_total,"98,891.4",4.5%
english_geog_vs_reg_total,"127,566.3",5.8%
racial_geog_vs_reg_total,"157,275.4",7.2%
racial_geog_vs_50_percent,"166,500.2",7.6%
poverty_geog_vs_reg_total,"136,134.1",6.2%
poverty_geog_vs_50_percent,"301,633.4",13.8%
disability_geog_vs_reg_total,"114,629.4",5.2%
NOT in youth_geog_vs_reg_total,"209,782.2",9.6%


## Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.

Average accessible jobs are weighted averages based on parcel household population.

In [ ]:
def bp_job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin(['region', geog])].\
        rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs (1-mile walk)'] = df['Jobs within 1-mile Walk'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')
    df['% Total Jobs (3-mile bike)'] = df['Jobs within 3-mile Bike'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')

    return df

In [ ]:
df = bp_job_access_geog(df_access_bp,'CountyName')

f_name = str(incr)+'_walk_bike_jobs_county_region.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
King,"18,519.3","84,914.7",0.8%,3.9%
Kitsap,"1,159.5","7,422.4",0.1%,0.3%
Outside Region,0.0,31.0,0.0%,0.0%
Pierce,"2,537.5","18,572.3",0.1%,0.8%
Snohomish,"2,035.6","17,994.2",0.1%,0.8%
Region,"11,172.5","54,353.7",0.5%,2.5%


In [ ]:
df_rgc = bp_job_access_geog(df_access_bp,'rgc_binary')
df = bp_job_access_geog(df_access_bp,'GrowthCenterName')

df = pd.concat([df_rgc, df.loc[~df.index.isin(['Region','Not in RGC'])]], axis=0)

f_name = str(incr)+'_walk_bike_jobs_rgc.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,"11,172.5","54,353.7",0.5%,2.5%
Not in RGC,"2,923.2","34,998.1",0.1%,1.6%
In RGC,"88,863.0","236,642.0",4.1%,10.8%
Auburn,"10,541.0","40,324.1",0.5%,1.8%
Bellevue,"59,188.0","110,699.5",2.7%,5.1%
Bothell Canyon Park,"8,538.5","21,748.3",0.4%,1.0%
Bremerton,"11,477.8","29,603.3",0.5%,1.4%
Burien,"4,828.8","13,399.8",0.2%,0.6%
Everett,"15,192.3","35,064.8",0.7%,1.6%


In [ ]:
df = bp_job_access_geog(df_access_bp,'rg_proposed')

f_name = str(incr)+'_walk_bike_jobs_reg_geog.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,"11,172.5","54,353.7",0.5%,2.5%
CitiesTowns,983.7,"8,947.9",0.0%,0.4%
Core,"3,416.4","28,646.4",0.2%,1.3%
HCT,"1,495.0","15,923.3",0.1%,0.7%
Metro,"29,413.7","127,845.4",1.3%,5.8%
UU,442.6,"7,024.1",0.0%,0.3%


In [ ]:
df = bp_job_access_geog(df_access_equity_bp,'Equity Geography')

f_name = str(incr)+'_walk_bike_jobs_equity.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
youth_geog_vs_reg_total,"1,735.5","19,321.5",0.1%,0.9%
elderly_geog_vs_reg_total,"8,543.6","40,332.7",0.4%,1.8%
english_geog_vs_reg_total,"10,839.3","48,404.3",0.5%,2.2%
racial_geog_vs_reg_total,"18,062.9","69,070.9",0.8%,3.2%
racial_geog_vs_50_percent,"18,652.8","70,870.7",0.9%,3.2%
poverty_geog_vs_reg_total,"13,811.3","57,046.9",0.6%,2.6%
poverty_geog_vs_50_percent,"30,973.6","101,410.4",1.4%,4.6%
disability_geog_vs_reg_total,"12,956.3","51,239.9",0.6%,2.3%
NOT in youth_geog_vs_reg_total,"18,909.0","83,073.6",0.9%,3.8%


## Intersection Density

In [9]:
buffered_parcels = pl.read_csv(output_path / 'landuse/buffered_parcels.txt', 
                               separator=' ',
                               columns=['parcelid','nodes3_2','nodes4_2','hh_p'])


async_engine = create_engine('sqlite:///' + summary_config['sc_run_path'] + '/inputs/db/' + config['db_name'])

list_cols = ['ParcelID','CountyName','GrowthCenterName','rg_proposed'] + equity_geogs
parcel_geog = pl.read_database(
    query= f"SELECT {', '.join(list_cols)} FROM " + "parcel_" + config["base_year"] + "_geography",
    connection=async_engine.connect()
)

In [18]:
df_intersection = buffered_parcels.join(parcel_geog, left_on='parcelid', right_on='ParcelID').to_pandas()

# Total intersections within 1/2 mile buffer
df_intersection['intersections_wt'] = (df_intersection['nodes3_2'] + df_intersection['nodes4_2']) * df_intersection['hh_p']

In [19]:
def intersection_density(geog):
    df = df_intersection.groupby(geog)[['intersections_wt', 'hh_p']].sum().reset_index()
    df['Intersections'] = df['intersections_wt']/df['hh_p']
    
    return df[[geog] + ['Intersections']]

In [ ]:
intersection_density('CountyName')

,CountyName,Intersections
0,King,155.0
1,Kitsap,50.6
2,Outside Region,3.6
3,Pierce,80.1
4,Snohomish,73.4


In [ ]:
intersection_density('GrowthCenterName')

,GrowthCenterName,Intersections
0,Auburn,192.4
1,Bellevue,273.3
2,Bothell Canyon Park,68.8
3,Bremerton,168.0
4,Burien,175.8
5,Everett,160.6
6,Federal Way,133.9
7,Greater Downtown Kirkland,166.5
8,Issaquah,NaN
9,Kent,214.0


In [ ]:
intersection_density('rg_proposed')

,rg_proposed,Intersections
0,CitiesTowns,64.6
1,Core,103.2
2,HCT,84.2
3,Metro,201.6
4,UU,56.1


In [ ]:
df_intersection_equity = df_intersection.loc[df_intersection['disability_geog_vs_reg_total']>=0].copy()

# get total intersections by equity geography
df1 = df_intersection_equity[equity_geogs].apply(lambda x: x * df_intersection_equity['intersections_wt']).sum().reset_index()
df1.columns = ['Equity Group', 'intersections_wt']

# get total households by equity geography
df2 = df_intersection_equity[equity_geogs].apply(lambda x: x * df_intersection_equity['hh_p']).sum().reset_index()
df2.columns = ['Equity Group', 'hh_p']

df_in = df1.merge(df2, on='Equity Group')
df_in['Inside Equity Geography'] = df_in['intersections_wt']/df_in['hh_p']

# outside equity geographies
df1_out = df_intersection_equity[equity_geogs].apply(lambda x: (1-x) * df_intersection_equity['intersections_wt']).sum().reset_index()
df1_out.columns = ['Equity Group', 'intersections_wt']
df2_out = df_intersection_equity[equity_geogs].apply(lambda x: (1-x) * df_intersection_equity['hh_p']).sum().reset_index()
df2_out.columns = ['Equity Group', 'hh_p']
df_out = df1_out.merge(df2_out, on='Equity Group')
df_out['Outside Equity Geography'] = df_out['intersections_wt']/df_out['hh_p']

df = df_in[['Equity Group','Inside Equity Geography']].merge(
    df_out[['Equity Group','Outside Equity Geography']], on='Equity Group')

df

,Equity Group,Inside Equity Geography,Outside Equity Geography
0,youth_geog_vs_reg_total,77.6,151.7
1,elderly_geog_vs_reg_total,100.9,134.2
2,english_geog_vs_reg_total,120.4,117.1
3,racial_geog_vs_reg_total,136.0,103.5
4,racial_geog_vs_50_percent,139.7,108.7
5,poverty_geog_vs_reg_total,127.1,112.5
6,poverty_geog_vs_50_percent,197.9,116.6
7,disability_geog_vs_reg_total,115.4,120.9
